In [89]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split


In [90]:
dataset = 'model/handClassifier/keypoint.csv'
model_save_path = 'model/handClassifier/keypoint_classifier.keras'
tflite_save_path = 'model/handClassifier/keypoint_classifier.tflite'

# Set number of classes

In [91]:
NUM_CLASSES = 5

# Dataset reading

In [92]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [93]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [94]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=42)

# Model building

In [95]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [96]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout_10 (Dropout)            │ (None, 42)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 20)             │           860 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 5)              │            55 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,125 (4.39 KB)

 Trainable params: 1,125 (4.39 KB)

 Non-trainable params: 0 (0.00 B)

In [97]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [98]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model training

In [99]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
 1/30 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.2031 - loss: 1.8545
Epoch 1: saving model to model/keypoint_classifier/keypoint_classifier.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.2452 - loss: 1.7344 - val_accuracy: 0.3424 - val_loss: 1.5163
Epoch 2/1000
 1/30 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.2969 - loss: 1.6022
Epoch 2: saving model to model/keypoint_classifier/keypoint_classifier.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3169 - loss: 1.5431 - val_accuracy: 0.3881 - val_loss: 1.4115
Epoch 3/1000
 1/30 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.3125 - loss: 1.4762
Epoch 3: saving model to model/keypoint_classifier/keypoint_classifier.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3523 - loss: 1.4291 - val_accuracy: 0.4330 - val_loss: 1.3060
Epoch 4/1000
 1/30 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.3359 - loss: 1.3740
Epoch 4: saving model to model/keypoint_classifier/keypoint_classifier.keras


In [100]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 891us/step - accuracy: 0.9295 - loss: 0.2912


In [101]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

In [102]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
[5.7717401e-01 3.0964047e-01 2.8086230e-03 7.8596393e-05 1.1029836e-01]
0


# Convert to model for Tensorflow-Lite

In [103]:
# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)

In [104]:
# Transform model (quantization)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

INFO:tensorflow:Assets written to: C:\Users\narte\AppData\Local\Temp\tmpoakigqp7\assets


INFO:tensorflow:Assets written to: C:\Users\narte\AppData\Local\Temp\tmpoakigqp7\assets


Saved artifact at 'C:\Users\narte\AppData\Local\Temp\tmpoakigqp7'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 42), dtype=tf.float32, name='input_layer_5')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  2842905385616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2842905387344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2842905388688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2842905386384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2842905389648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2842905387152: TensorSpec(shape=(), dtype=tf.resource, name=None)


6600

# Inference test

In [105]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [106]:
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [107]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [108]:
%%time
# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

CPU times: total: 0 ns
Wall time: 0 ns


In [109]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[5.77173948e-01 3.09640437e-01 2.80862395e-03 7.85965385e-05
 1.10298395e-01]
0
